In [2]:
import random
import cv2
import math
import imageio

MAX_NUMBER_OF_FRAMES = 100

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 10

NUMBER_OF_NP_CANDIDATES = 10

# Location to read images from.
# NOTE: "_" is used to follow a naming convention
# eg. _20.jpg, _21.jpg etc...
READ_LOCATION = "/Users/okanbagriacik/Desktop/Kareler/_"

# Location to write GIF images to.
GIF_WRITE_LOCATION = "/Users/okanbagriacik/Desktop/GIFs/sample2.GIF"

# Population matrix.
NP = []

# Mutation vector.
MV = []

# Trail vector.
TV = []

# Scale factor.
F = 0.9

# Cr probability value.
Cr = 0.6

# Calculate AED for a chromosome.
def getAED( KF ):
    ED_sum = 0
    for i in range(1, TOTAL_KEY_FRAMES - 1):
        while True:
            try:
                im1 = cv2.imread(READ_LOCATION + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(READ_LOCATION + str(KF[i+1]) + ".jpg",0)
                ED_sum += cv2.norm(im1, im2, cv2.NORM_L2)
            except:
                print (i, KF, KF[i], KF[i+1])
                continue
            break
    return ED_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and AEDs).
def initialize_NP():
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(1, MAX_NUMBER_OF_FRAMES+1), TOTAL_KEY_FRAMES)))
        NP[-1].append(getAED(NP[-1]))
        print (NP[-1])

# MUTATION
def mutation(num):
    R = random.sample(NP,3)
    global MV
    MV[:] = []
    MV_value = 0
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[num][i] + F*(R[1][i] - R[2][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > MAX_NUMBER_OF_FRAMES):
            MV.append(MAX_NUMBER_OF_FRAMES)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getAED(MV))

# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant):
    print ("mutant: ", mutant)
    print ("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getAED(TV))
    print ("TV    : ", TV)

# SELECTION : Selects offspring / parent based on higher ED value.
def selection(parent, trail_vector):
    if(trail_vector[-1] > parent[-1]):
        parent[:] = trail_vector
        print ("yes", parent)
    else:
        print ("no")

# bestParent returns the parent with then maximum ED value.
def bestParent(population):
    Max_AED_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] > Max_AED_value):
            Max_AED_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index




initialize_NP()
for GENERATION in range(STOPPING_ITERATION):
    for i in range(NUMBER_OF_NP_CANDIDATES):
        print ("---------------------", "PARENT:", i+1 , "GENERATION:", GENERATION+1, "---------------------")
        mutation(i)
        crossover(NP[i], MV)
        selection(NP[i], TV)
        print (NP[i])
        TV[:] = []
        print ("")
    print ("")
best_parent = bestParent(NP)
print ("best solution is: ", best_parent)
images_for_gif = []
for frame_number in best_parent[:-1]:
        images_for_gif.append(imageio.imread(READ_LOCATION + str(frame_number) + '.jpg'))
imageio.mimsave(GIF_WRITE_LOCATION, images_for_gif)

[5, 10, 16, 33, 53, 66, 81, 94, 95, 98, 41411.87198691259]
[1, 5, 7, 19, 23, 35, 38, 65, 88, 89, 35644.65309505883]
[13, 25, 57, 58, 59, 63, 68, 76, 83, 95, 41359.75063040601]
[16, 21, 22, 24, 26, 45, 46, 59, 66, 74, 34359.23435735968]
[13, 24, 29, 34, 50, 69, 79, 85, 93, 99, 40450.05376994729]
[12, 26, 40, 43, 52, 62, 63, 65, 78, 89, 35928.526502536115]
[10, 21, 32, 40, 43, 53, 54, 67, 77, 98, 40007.48588011491]
[7, 8, 9, 24, 28, 38, 46, 63, 73, 80, 39761.492134468404]
[8, 15, 17, 21, 32, 45, 61, 64, 72, 79, 40897.638249915966]
[2, 5, 29, 35, 42, 65, 72, 80, 81, 96, 38632.8753097806]
--------------------- PARENT: 1 GENERATION: 1 ---------------------
mutant:  [12, 23, 52, 55, 58, 63, 69, 77, 84, 95, 47200.158806269734]
parent:  [5, 10, 16, 33, 53, 66, 81, 94, 95, 98, 41411.87198691259]
TV    :  [10, 12, 16, 33, 53, 66, 69, 94, 95, 95, 38556.5283493903]
no
[5, 10, 16, 33, 53, 66, 81, 94, 95, 98, 41411.87198691259]

--------------------- PARENT: 2 GENERATION: 1 ---------------------
mut